# O Problema / The Problem

## O Problema

Uma empresa varejista coletou dados de 1000 clientes que realizaram compras online em diversas partes do Brasil.

A ideia da companhia é conhecer melhor o perfil de seus clientes a
fim de oferecer novos itens, promoções e alavancar as vendas.

A equipe de cientistas de dados analisou o caso e identificou que, para a realização dessa tarefa, é necessário realizar um agrupamento dos dados para realizar a segmentação dos clientes.

**Lista de atividades**
1. Coleta de dados;
2. Análise dos dados coletados;
3. Realizar o tratamento de dados;
4. Manipulação e visualização de dados;
5. Aplicação de algoritmo de Machine Learning k-means;
6. Análise de dados gerados;
7. Realizar a segmentação dos clientes.

----
## The Problem

A retail company have collected data from 1000 customers who made online purchases in different parts of Brazil.

The company's idea is to better understand the profile of its customers
in order to offer new items, promotions and to boost its sales.

The Data Scientists team analyzed the case and identified that, in order to carry out this task, it is necessary to group the data to perform the segmentation of customers.

**Activities list**
1. Data collection;
2. Analysis of the collected data;
3. Carry out data processing;
4. Data manipulation and visualization;
5. Application of k-means Machine Learning algorithm;
6. Analysis of the generated data;
7. Perform customer segmentation.


# **1.** Coleta de Dados e Iniciando Bibliotecas - Data Collection and Initiating Libraries

## Bibliotecas necessárias / Needed Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns                     #For Statistical data visualization as boxplot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
from sklearn.cluster import KMeans        #Machine Learning library for Python. This one specifically: loads KMeans

# Este Jupyter Notebook foi criado inicialmente usando
# This Jupyter Notebook was initially created running
# pandas version 1.3.5
# numpy version  1.22.0
# plotly version 5.10.0
# sklearn version 1.0.2

# Se você gostaria de checar as versões do plotly e do sklearn instalados, por favor, retire o comentário (#) das duas últimas linhas abaixo.
# If you want to check the version of plotly and skylearn, please uncomment the last two lines below
# import plotly
# import sklearn

In [ ]:
print(f''' Verificando as versões das bibliotecas usadas / Verifying currently used Libraries versions: 
pandas: {pd.__version__}
numpy: {np.__version__}
''')

## Coleta de Dados / Data Collection

In [ ]:
# Dados dos clientes (arquivo em xlsx) / Customer data (.xlsx file)
customer_data = pd.read_excel('./clientes.xlsx')

# Dados dos estados brasileiros (arquivo em csv) / Brazilian States data (.csv file)
states = pd.read_csv('./estados_brasileiros.csv', sep=';', encoding = 'latin-1')

# Dados complementares dos clientes / Additional Customer data
customer_data_add = pd.read_csv('./dados_complementares.csv', sep = ';', encoding = 'latin-1')


# **2.** Análise dos Dados Coletados / General Analysis of the collected data

In [ ]:
#2.1 Ver a primeira entrada para de cada dado coletado
#2.1 See the first entry from each collected data
customer_data[:1]

In [ ]:
states[:1]

In [ ]:
customer_data_add[:1]

In [ ]:
#2.2: Usando a função info(). Objetivo: ver tipo dos dados e dados faltantes
#2.2: Using the info() function. Objective: see the data types and missing data
customer_data.info()

In [ ]:
#2.3 Usando a função isna().sum(). Objetivo: ver a quantidade total de dados faltantes. 0 se não há dados faltantes.
#2.3 Using the function isna().sum(). Objetive: see the total amount of missing data. 0 if none.

customer_data.isna().sum()

1. Existem 11 dados faltando na coluna de `idade`. `cod_cliente`, `cod_genero` são tipo inteiro.
2. `genero` é tipo texto (object).
3. `idade` está como float.
----
1. There are 11 missing data in `idade` (age) column.
2. `cod_cliente`, `cod_genero` are integers.
3. The type of `genero` (gender) is string (object).

In [ ]:
customer_data_add.info()

In [ ]:
customer_data_add.isna().sum()

1. Existem 6 dados faltando na coluna de `salario`, e 13 na coluna de `compras_3M`. 
2. `cod_cliente`, `aposentado`, `plano_saude`, `qnt_filhos` e `cod_estado` são do tipo inteiro. 
3. `salario` e `compras_3M` (compra feita nos últimos 3 meses) são float.
----
1. There are 6 missing data in `salario` (salary), and 13 in `compras_3M`. 
2. `cod_cliente` (customer number), `aposentado` (retired), `plano_saude` (health care), `qnt_filhos` (number of children) and `cod_estado` (State number) are integers. 
3. `salario` and `compras_3M` (purchase made in the last 3 months) are float.

In [ ]:
states.info()

1. Não existem dados faltantes.
2. `estado`, `sigla_estado` e `pais` são tipo texto (object).
----
1. There is no missing data.
2. `estado` (State), `sigla_estado` (State Acronym) e `pais` (Country) are all strings (object).

In [ ]:
#2.4 Usando describe(). Objetivo: ver um resumo estatístico dos dados coletados
#2.4 Using describe(). Objective: see a statistical summary of the collected data 
customer_data.describe()

Dos dados acima, uma vez que `cod_genero` associa os valores 0 para sexo masculino ou 1 para sexo feminino, pela média de `cod_genero`, temos 49,2% de clientes do sexo feminino e 50,8% do sexo masculino.

----

From the data above, since `cod_genero` takes the values 0 to masculine sex and 1 to feminine sex, seeing the `cod_genero` mean, we have 49,2% of customers of the feminine sex and 50,8% of customers of the masculine sex in this database.


In [ ]:
customer_data_add.describe()

Agora vemos que 51,4% das pessoas neste banco de dados possuem um plano de saúde (`plano_saude`=1).

----
Now we see that 51,4% of the people in this database have a health care plan (`plano_saude` = 1).

# **3.** Tratamento dos Dados Faltantes / Missing Data Processing

Nas instruções do serviço nos é dito que:
* Não existem compras nos últimos três meses para registros nulos.
* Se houver dados de idade/quantidade de filhos faltando, trocar pela média da idade/quantidade de filhos do gênero do/da cliente.
* Na ausência de dados de salário, se aposentado ou não, ou se possui plano de saúde ou não, eliminar os registros.
* Para outras variáveis categóricas, utilizar a correção pela moda da variável

Para os nossos dados, essas condições são equivalentes a:
1. Tomar como **0** todos os dados nulos em `'compras_3M'` .
2. Para os dados faltantes em "idade", trocar os dados faltantes pela média do gênero do/da cliente.
3. Apagar todos os registros de `'salario'` faltantes.

----
In the service instructions we are told that:
* Null registrations for in the last three months means that there were no purchases in this period.
* If age/number of children info are missing, replace them by the average age/number of children of the customer's sex.
* In the absence of data about salary, whether retired or not, or whether having health insurance or not, delete the records.
* For other categorical variables, correct the data using the mode of the variable

In the current data, all of these are equivalent to:
1. Setting **0** to all null `'compras_3M'`.
2. For the missing age data (`'idade'`), change the missing data considering the customer's sex.
3. Deleting all the missing records of `'salario'`.

In [ ]:
#3.1: Vamos tomar 0 todos os dados nulos em "compras_3M"
#3.1: Let's set 0 to all null data in "compras_3M"
customer_data_add.compras_3M.fillna(0,inplace=True)

#3.2: Vamos apagar todos os dados nulos em "salario"
#3.2: Let's drop all null data in "salario"
customer_data_add.dropna(subset=['salario'], inplace=True)
customer_data_add.isna().sum()

In [ ]:
#3.3: Vamos encontrar a média de idades por gênero primeiro. 
#   Definimos uma variável média para tais dados. Tornamos essas variáveis como inteiras.
#   Finalmente preenchemos os dados vazios por tais médias

#3.3: Let's first find the mean age by gender sex. 
#   Then, we define variables called mean for each data. We turn these variables to integer numbers.
#   Finally we fill the missing data with the mean found.

mean_age = customer_data.groupby('genero')['idade'].mean()

mean_age

In [ ]:
mean_age_feminine = int(mean_age['feminino']) #Média de idade sexo feminino / Mean for the feminine sex
mean_age_masculine = int(mean_age['masculino'])  #Média de idade sexo masculino/ Mean for the masculine sex

In [ ]:
print(f' Idade média Sexo Feminino: {mean_age_feminine}\n Idade média Sexo Masculino: {mean_age_masculine}')
print(f' Mean age of Feminine sex: {mean_age_feminine}\n Mean age of Masculine sex: {mean_age_masculine}')

In [ ]:
customer_data.loc[(customer_data['genero']=='masculino') & (customer_data['idade'].isnull()),'idade'] = mean_age_masculine
customer_data.loc[(customer_data['genero']=='feminino') & (customer_data['idade'].isnull()),'idade'] = mean_age_feminine

In [ ]:
customer_data.isna().sum()

# **4.** Manipulação e Visualização de Dados / Data Manipulation and Visualization

## 4.1 - Manipulação de Dados / Data Manipulation

In [ ]:
#4.1: Vamos primeiro ver quais indexadores são comuns de cada banco de dados
#4.2: Let's first check which indices are common to each database

In [ ]:
customer_data.columns

In [ ]:
customer_data_add.columns

In [ ]:
states.columns

Acima nós percebemos que
* `customer_data` possui `'cod_cliente'` em comum com `customer_data_add`
* `customer_data_add` possui `'cod_estado'` em comum com `states`.

----
From above, we notice that
* both `customer_data` and `customer_data_add` have `'cod_cliente'`
* both `customer_data_add` and `states` have `'cod_estado'`.

In [ ]:
#4.2: Vamos unir os dados de customer_data e customer_data_add a um novo conjunto de dados chamado all_data_customer.
#     Então, vamos adicionar a este conjunto, os dados em states. Chamaremos este de customer_full_data

#4.2: Let's first join the data in customer_data and customer_data_add to a new dataframe called all_data_customer.
#     Then, let's add to this dataframe the data in states, and call it customer_full_data

all_data_customer = pd.merge(customer_data,customer_data_add, on= 'cod_cliente').sort_values('cod_cliente')

all_data_customer.columns

In [ ]:
customer_full_data = pd.merge(all_data_customer,states, on= 'cod_estado').sort_values('cod_cliente')

In [ ]:
# Agora que criamos nosso banco de dados principal, vamos apagar as variáveis extras e não usadas daqui por diante.
## Essa prática é útil para gerenciamento de memória.

# Now that we have created our main database, we will delete all the unused variables we have created so far.
## This is a good practice for memory management.
del all_data_customer, customer_data_add, customer_data, mean_age, mean_age_feminine, mean_age_masculine

In [ ]:
customer_full_data.head(3)

In [ ]:
#4.3: Finalmente, vamos fazer uma seleção das colunas neste novo dataset. Logo em seguida resetamos os valores dos indices do dataframe com reset_index(drop=True)
#4.3: Finally, let's do a selection of which columns we want in the dataset. Right after, we reset the datafram index values with com reset_index(drop=True)

column_order = ['cod_cliente','genero','idade','salario','aposentado','plano_saude','qtd_filhos','compras_3M','estado','sigla_estado']
customer_full_data = customer_full_data[column_order].reset_index(drop=True)
del column_order

In [ ]:
customer_full_data.head(3)

In [ ]:
# 4.4: Agora, para uma melhor visualização dos dados, vamos passar alguns dados 1/0 para Sim/Não.
#     Adicionalmente, vamos criar uma coluna para região do país: Norte, Nordeste, Sul, Sudeste e Centro-Oeste.

# 4.4: Now, to a better data look, we are going to change the indices with values 1/0 to 'Sim' (Yes) / 'Não' (No).
#     Additionally, we are going to create an new column labelling each Brazilian State to its region:
#     North ('Norte), South ('Sul'), Northest ('Nordeste'), Southest ('Sudeste') and Midwest ('Centro-Oeste')

customer_full_data.loc[customer_full_data['aposentado']==1,'aposentado'] = 'Sim'
customer_full_data.loc[customer_full_data['aposentado']==0,'aposentado'] = 'Não'
customer_full_data.loc[customer_full_data['plano_saude']==0,'plano_saude'] = 'Não'
customer_full_data.loc[customer_full_data['plano_saude']==1,'plano_saude'] = 'Sim'
customer_full_data.loc[customer_full_data['compras_3M']==1.0,'compras_3M'] = 'Sim'
customer_full_data.loc[customer_full_data['compras_3M']==0.0,'compras_3M'] = 'Não'

regiao_norte = ['AC','AP','AM','PA','RO','RR','TO']
regiao_nordeste = ['CE','MA','BA','PE','PB','RN','PI','AL','SE']
regiao_centroeste = ['MT','MS','GO','DF']
regiao_sudeste = ['RJ','SP','MG','ES']
regiao_sul = ['SC','PR','RS']
customer_full_data.loc[customer_full_data['sigla_estado'].isin(regiao_norte),'Região'] = 'Norte'
customer_full_data.loc[customer_full_data['sigla_estado'].isin(regiao_nordeste),'Região'] = 'Nordeste'
customer_full_data.loc[customer_full_data['sigla_estado'].isin(regiao_sudeste),'Região'] = 'Sudeste'
customer_full_data.loc[customer_full_data['sigla_estado'].isin(regiao_sul),'Região'] = 'Sul'
customer_full_data.loc[customer_full_data['sigla_estado'].isin(regiao_centroeste),'Região'] = 'Centro-Oeste'

In [ ]:
customer_full_data.tail()

## 4.2 - Visualização de Dados / Data Visualization

In [ ]:
#4.6: Checando correlação entre os dados
#4.6: Checking correlation between all the data

plt.figure(figsize=(10,6))
plt.title('Correlação entre os dados / Data Correlation')
dados_corr = customer_full_data[['idade','salario','qtd_filhos']]
sns.heatmap(dados_corr.corr(), cmap='crest',linewidths=0.1,linecolor='white',annot=True, cbar=False, vmin=-1, vmax = 1)
sns.set_context('notebook', font_scale=1.44)

In [ ]:
del dados_corr

A partir do gráfico acima, vemos pouca correlação entre idade, salário e quantidade de filhos.

----
From the plot above, we see little correlation bewteen age ['idade'], salary ['salario'] and number of children ['qtd_filhos'].

In [ ]:
plt.figure(figsize=(15,10))
plt.title('Salários por Estados / Wage by States ')
plot1 = sns.boxplot(x= 'salario', y= 'estado', data=customer_full_data.sort_values('estado'), orient='h')
plot1.set(xlabel ='Salário/Salary', ylabel = 'Estados/States')
sns.set_context('notebook', font_scale=1.17)

In [ ]:
plt.figure(figsize=(15,10))
plt.title('Idade por Estado / Age by State ')
plot1 = sns.boxplot(x= 'idade', y= 'estado', data=customer_full_data.sort_values('estado'), orient='h')
plot1.set(xlabel= 'Idade/Age', ylabel= 'Estados/States')
sns.set_context('notebook', font_scale=1.17)

In [ ]:
plt.figure(figsize=(15,10))
plt.title('Salários por Região / Salary by Region ')
plot1 = sns.boxplot(x= 'salario', y= 'Região', data=customer_full_data.sort_values('Região'), orient='h')
plot1.set(xlabel ='Salário/Salary', ylabel = '')
sns.set_context('notebook', font_scale=1.17)

In [ ]:
plt.figure(figsize=(15,10))
plt.title('Idade por Região / Age by Region ')
plot1 = sns.boxplot(x= 'idade', y= 'Região', data=customer_full_data.sort_values('Região'), orient='h')
plot1.set(xlabel= 'Idade/Age', ylabel= '')
sns.set_context('notebook', font_scale=1.17)

In [ ]:
idade_byGenero_processed = customer_full_data.query('genero == "masculino"').reset_index(drop=True)
plt.figure(figsize=(12,7))
plt.title('Idade dos clientes do sexo masculino / Customers age of masculine sex')
plt.xlabel('Idade / Age')
plt.ylabel('Quantidade de clientes / Amount of customers')
sns.histplot(idade_byGenero_processed.idade, bins=20)
sns.set_context('notebook')

In [ ]:
plt.figure(figsize=(28,5))
plt.title('Distribuição dos Clientes por Sexo e Estado / Customer Distribution by Gender Sex and States')
plot_GenderSex_state = sns.countplot(x='sigla_estado', data = customer_full_data.sort_values('sigla_estado'), hue='genero', palette='Set1')
plot_GenderSex_state.set(xlabel = 'Estados / States', ylabel = 'Quantidade de clientes / Amount of customers')

In [ ]:
# É difícil ler o gráfico acima com tantas informações de estados, então agora fazemos o mesmo gráfico, porém por região
# It is quite hard to see the above plot, so now we plot it by Brazilian region
plt.figure(figsize=(20,5))
plt.title('Distribuição dos Clientes por Sexo e Região / Customer Distribution by Gender Sex and Region')
plot_GenderSex_state = sns.countplot(x='Região', data = customer_full_data.sort_values('Região'), hue='genero', palette='Set1')
plot_GenderSex_state.set(xlabel = 'Regiões / Regions', ylabel = 'Quantidade de clientes / Amount of customers')
sns.set_context('talk')

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Distribuição dos Clientes por Sexo e Região / Customer Distribution by Gender Sex and Region')
plot_GenderSex_state = sns.histplot(x='Região', data = customer_full_data.sort_values('Região'), hue='genero', palette='Set1',multiple='dodge', shrink = .7)
plot_GenderSex_state.set(xlabel = 'Regiões / Regions', ylabel = 'Quantidade de clientes / Amount of customer')
plt.legend(title='Gênero/Gender', loc='upper right', labels=['Feminino', 'Masculino'])
sns.set_context('talk')

# **5.** Aplicação do Algoritmo K-Means / Using K-Means Algorithm

## 5.1 - Normalização dos dados / Normalizing the data

### 5.1.1 - WCSS e o Método do Cotovelo / WCSS and the Elbow Method

In [ ]:
#5.1: Para normalizar os dados, precisamos importar a biblioteca standard scale
#5.1: In order to normalize the date we need to import the standard scale library
from sklearn.preprocessing import StandardScaler

idade_salario = customer_full_data.iloc[:,[2,3]].values
idade_salario_normal = StandardScaler().fit_transform(idade_salario)

In [ ]:
#5.1: Definindo uma função calculate_wcss tendo como entrada idade_salario_normal dos clientes
#5.1: Defing a new function calculate_wcss taking customer data from idade_salario_normal as entries.

def calculate_wcss(data_client):
  wcss = []
  for k in range(1,11):
    kmeans = KMeans(n_clusters= k, random_state=0)
    kmeans.fit(X=data_client)
    wcss.append(kmeans.inertia_)
  return wcss

In [ ]:
#5.2: Calculando o valor de wcss para cada número de clusters.
#5.2: Calculating the wcss for each number of clusters
wcss_customer = calculate_wcss(idade_salario_normal)

In [ ]:
for i in range(len(wcss_customer)):
  print(f'O cluster {i} possui wcss de {wcss_customer[i]} / The wcss value for the cluster {i} is {wcss_customer[i]}')

In [ ]:
#5.3: Gráfico do WCSS por número de clusters dos dados normalizados
#5.3: Plot of the WCSS values by number of clusters of the normalized data

grafico_wcss = px.line(x= range(1,11),y=wcss_customer)
fig = go.Figure(grafico_wcss)
fig.update_layout(title = 'Análise do Cotovelo de WCSS / The WCSS Elbow Analysis',
                  title_x = 0.5,
                  xaxis_title = 'Número de Clusters / Number of Clusters',
                  yaxis_title = 'WCSS',
                  template='plotly_dark', width=1000)
fig.add_annotation(x=4, y=wcss_customer[4-1],
            text="Cotovelo / Elbow point",
            showarrow=True,
            arrowhead=2)
fig.show("png") 
# Eu adicionei a opção "png" para que o gráfico seja exibido no ambiente estático do GitHub. Remova "png" se você quiser vê-lo como um gráfico Plotly normal
# I've add the "png" option to showcase it on static environment from GitHub. Remove if you want to see it as a normal Plotly graph.

O melhor valor para o número de clusters pelo método do cotovelo é 4.

----
The best number for Clusters by the elbow Method is 4.

### 5.1.2 - Agrupando os dados com o uso do KMeans / Clustering the Data with KMeans

In [ ]:
#5.4: Agora nós nos focamos no algoritmo KMeans com 4 clusters.
#5.4: We now focus on the KMeans with 4 clusters.

kmeans_customer = KMeans(n_clusters = 4, random_state=0)
customer_full_data['cluster'] = kmeans_customer.fit_predict(idade_salario_normal)
#Estamos adicionando uma nova coluna a customer_full_data que toma a qualidade de pertencer a um cluster.
#We are adding a new column to customer_full_data that takes quality of belonging to a cluster.


In [ ]:
label_cluster_customer_age_wage = kmeans_customer.fit_predict(idade_salario_normal)
#Variável necessária somente para comparação dos resultados com os dados normalizados e naturais.
#Variable necessary only to compare with the results using normalized data and natural data.


In [ ]:
#5.5: Calculando os centróides dos clusters.
#5.5: Calculating the cluster centers.

centroide_clusters = kmeans_customer.cluster_centers_
centroide_clusters

In [ ]:
#Como podemos ver abaixo, agora temos uma nova coluna no final de customer_full_data chamada cluster.
#As we can see below, now at the end we have a new column called cluster.
customer_full_data.columns

In [ ]:
#5.6: Abaixo traduzimos os valores dos centróides das variáveis normalizada para seus valores naturais
#5.6: Below we translate the cluster centers values from the normalized data to its natural values
import copy

centroide_clusters_Natural = copy.deepcopy(centroide_clusters)
centroide_clusters_Natural[:,0] = centroide_clusters_Natural[:,0]*customer_full_data.idade.std()+customer_full_data.idade.mean()
centroide_clusters_Natural[:,1] = centroide_clusters_Natural[:,1]*customer_full_data.salario.std()+customer_full_data.salario.mean()
centroide_clusters_Natural

In [ ]:
centroide_clusters

### 5.1.3 (extra) : Usando a inicialização `k-keans++` / Using the `k-means++` Inicialization

In [ ]:
#Nesta subseção, vamos apenas utilizar a inicialização do k-means++.
#In this sub-section, we are going to just do the same thing as before, but we are going to use the init = 'k-means++'

kmeans_customer_kmPP = KMeans(n_clusters = 4, init='k-means++' ,random_state=0)
#customer_full_data['cluster'] = kmeans_customer.fit_predict(idade_salario_normal)

label_cluster_customer_age_wage_kmPP = kmeans_customer_kmPP.fit_predict(idade_salario_normal)

centroide_clusters_kmPP = kmeans_customer_kmPP.cluster_centers_
centroide_clusters_kmPP

# Como pode ser visto, as posições dos contróides em 'centroide_clusters' são as mesma das obtidas anteriormente.

# As it can be seem, the 'centroide_clusters' positions are the same as before

# **6.** Análise de Dados e Segmentação dos Clientes / Data Analysis and Customer Segmentation

## 6.1 - Alguns Gráficos com Segmentação de Clientes /  Some Plots with Customer Segmentation

In [ ]:
grafico = px.scatter(customer_full_data, x='idade', y='salario', color = 'cluster', symbol ='cluster')
grafico_centroide = px.scatter(x= centroide_clusters_Natural[:,0],
                               y=centroide_clusters_Natural[:,1],
                               size = [1,1/2,1/3,1/4])
fig = go.Figure(data = grafico.data + grafico_centroide.data)
fig.update_layout(title = 'Idade x Salário classificados em 4 clusters / Age x Salary classified in 4 clusters',
                  title_x = 0.5,
                  xaxis_title = 'Idade/Age',
                  yaxis_title = 'Salário/Salary',
                  template = 'plotly_white',
                  showlegend = False,
                  coloraxis_showscale= False)
fig.show("png")
# Eu adicionei a opção "png" para que o gráfico seja exibido no ambiente estático do GitHub. Remova "png" se você quiser vê-lo como um gráfico Plotly normal
# I've add the "png" option to showcase it on static environment from GitHub. Remove if you want to see it as a normal Plotly graph.

In [ ]:
#Mesmo do gráfico acima, porém com dados normalizados
#Same as above, but in normalized data
grafico = px.scatter(x= idade_salario_normal[:,0], y=idade_salario_normal[:,1], color = label_cluster_customer_age_wage, symbol = label_cluster_customer_age_wage)
grafico_centroide = px.scatter(x= centroide_clusters[:,0],
                               y=centroide_clusters[:,1],
                               size = [1,1/2,1/3,1/4])
fig = go.Figure(data = grafico.data + grafico_centroide.data)
fig.update_layout(title = 'Idade x Salário classificados em 4 clusters / Age x Salary classified in 4 clusters',
                  title_x = 0.5,
                  xaxis_title = 'Idade/Age',
                  yaxis_title = 'Salário/Salary',
                  template = 'plotly_white',
                  showlegend = False,
                  coloraxis_showscale= False)
fig.show("png")
# Eu adicionei a opção "png" para que o gráfico seja exibido no ambiente estático do GitHub. Remova "png" se você quiser vê-lo como um gráfico Plotly normal
# I've add the "png" option to showcase it on static environment from GitHub. Remove if you want to see it as a normal Plotly graph.

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Gráfico de Caixas segmentando os clientes por idade, dependendo do identidicação de seu cluster\nBox Plot segmenting the customers by age depending on their cluster identification')
plot1 = sns.boxplot(x= 'idade', y= 'cluster', data=customer_full_data, orient='h')
plot1.set(xlabel ='Idade / Age', ylabel = 'Cluster ID')
sns.set_context('notebook', font_scale=1.17)

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Gráfico de Caixas segmentando os clientes por salário, dependendo do identidicação de seu cluster\nBox Plot segmenting the customers by salary depending on their cluster identification')
plot1 = sns.boxplot(x= 'salario', y= 'cluster', data=customer_full_data, orient='h')
plot1.set(xlabel ='Salário / Salary', ylabel = 'Cluster ID')
sns.set_context('notebook', font_scale=1.17)

In [ ]:
customer_full_data.groupby('cluster')['salario'].describe()

In [ ]:
7110.345455 > 3713.553265+3315.829596

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Gráfico de Caixas segmentando os clientes por quantidade de filhos, dependendo do identidicação de seu cluster\nBox Plot segmenting the customers by amount of children depending on their cluster identification')
plot1 = sns.boxplot(x= 'qtd_filhos', y= 'cluster', data=customer_full_data, orient='h')
plot1.set(xlabel ='Número de Filhos / Amount of Children', ylabel = 'Cluster ID')
sns.set_context('notebook', font_scale=1.17)

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Distribuição dos Clientes por Sexo e Cluster ID / Customer Distribution by Gender Sex and Cluster ID')
plot_GenderSex_cluster = sns.histplot(x='cluster', data = customer_full_data.sort_values('cluster'), hue='genero', palette='Set1',multiple='dodge', shrink = .7, discrete = True)
plot_GenderSex_cluster.set(xlabel = 'Clusters', ylabel = 'Quantidade de clientes / Amount of customer')
plot_GenderSex_cluster.set_xticks(range(4))
plt.legend(title='Gênero/Gender', loc='upper right', labels=['Feminino', 'Masculino'])
sns.set_context('talk')

In [ ]:
customer_full_data.groupby(['genero','cluster'])['cluster'].describe()

In [ ]:
customer_full_data.groupby(['estado'])['salario'].mean()

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Distribuição dos Clientes Aposentado ou Não e Cluster ID / Customer Distribution If Retired or Not and Cluster ID')
plot_GenderSex_cluster = sns.histplot(x='cluster', data = customer_full_data.sort_values('cluster'), hue='aposentado', palette='Set1',multiple='dodge', shrink = .7, discrete = True)
plot_GenderSex_cluster.set(xlabel = 'Clusters', ylabel = 'Quantidade de clientes / Amount of customer')
plot_GenderSex_cluster.set_xticks(range(4))
plt.legend(title='Aposentado?', loc='upper right', labels=['Sim', 'Não'])
sns.set_context('talk')

## 6.2 - Outras Análises Interessantes (A maioria por questões) / Other Interesting Analysis (Mostly by the Questions)

In [ ]:
customer_full_data.groupby(['cluster','aposentado'])['aposentado'].describe()

In [ ]:
customer_full_data.groupby(['aposentado'])['estado'].describe()

In [ ]:
customer_full_data.groupby(['cluster','genero'])['genero'].describe()

In [ ]:
customer_full_data.groupby(['plano_saude'])['estado'].describe()

In [ ]:
customer_full_data.groupby(['compras_3M','aposentado','cluster'])['genero'].describe()

In [ ]:
customer_full_data.groupby(['cluster','genero'])['qtd_filhos'].describe()